In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import gc


import warnings
warnings.simplefilter('ignore')

In [2]:
train = pd.read_csv('training_dataset/hourly_dataset.csv')
train['time'] = pd.to_datetime(train['time'])

In [3]:
train1 = train[(train['time']>='2022-01-01 01:00:00')&(train['time']<'2022-05-01 01:00:00')].reset_index(drop=True)
# test1 = train[train['train or test']=='test1'].reset_index(drop=True)

train2 = train[(train['time']>='2022-05-08 01:00:00')&(train['time']<'2022-06-01 01:00:00')].reset_index(drop=True)
# test2 = train[train['train or test']=='test2'].reset_index(drop=True)

train3 = train[(train['time']>='2022-06-08 01:00:00')&(train['time']<'2022-07-21 01:00:00')].reset_index(drop=True)
# test3 = train[train['train or test']=='test3'].reset_index(drop=True)

train4 = train[(train['time']>='2022-07-28 01:00:00')&(train['time']<'2022-08-21 01:00:00')].reset_index(drop=True)
# test4 = train[train['train or test']=='test4'].reset_index(drop=True)

In [4]:
print(train1.shape, train2.shape, train3.shape, train4.shape)

(2880, 22) (576, 22) (1032, 22) (576, 22)


In [5]:
## 将小于零的值替换为空值
for df in tqdm([train1,train2,train3,train4]):
    for i in range(1,21):
        df.loc[df[f'flow_{i}']<0,f'flow_{i}'] = np.nan
        
        
## 将异常值替换为空值
for df in tqdm([train1,train2,train3,train4]):
    for i in range(1,21):
        df[f'flow_{i}_shift'] = df[f'flow_{i}'].shift(-1)
        for index,row in df.iterrows():
            if index != df.shape[0]-1:
                if pd.isna(row[f'flow_{i}_shift']) and pd.notna(row[f'flow_{i}']):
                    df.loc[index,f'flow_{i}'] = np.nan
        del df[f'flow_{i}_shift'];gc.collect

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
## 缺失值填充 --> 参考了 "简单策略0.49 Baseline"
## 用前一天的值进行填充
for i in range(1,5):
    locals()[f'train{i}'] = locals()[f'train{i}'].set_index('time')
    
for df in [train1,train2,train3,train4]:
    for date in tqdm(df.index):
        for col in range(20):
            period1 = pd.Timedelta(hours = 24)
            if pd.isna(df.loc[date][col]): #判断是否为空值
                if date - period1 in df.index: #如果该索引存在 （考虑到一下flow一开始就存在空值）
                    period2 = date - period1
                    while pd.isna(df.loc[period2][col]):
                        period2 = period2 - period1
                    df.loc[date, f'flow_{col+1}'] = df.loc[period2, f'flow_{col+1}']

                else: #对于flow一开始就存在缺失值的，用后一天的值进行填充
                    period2 = date + pd.Timedelta(hours = 24)
                    while pd.isna(df.loc[period2][col]):
                        period2 = period2 + period1
                    df.loc[date, f'flow_{col+1}'] = df.loc[period2, f'flow_{col+1}']

                
for i in range(1,5):
    locals()[f'train{i}'] = locals()[f'train{i}'].reset_index()

  0%|          | 0/2880 [00:00<?, ?it/s]

  0%|          | 0/576 [00:00<?, ?it/s]

  0%|          | 0/1032 [00:00<?, ?it/s]

  0%|          | 0/576 [00:00<?, ?it/s]

In [7]:
sub = pd.read_csv('sample_submission.csv')

sub1 = (train1.iloc[-24*7*1:,1:-1].values + train1.iloc[-24*7*2:-24*7*1,1:-1].values + \
        train1.iloc[-24*7*3:-24*7*2,1:-1].values + train1.iloc[-24*7*4:-24*7*3,1:-1].values)/4
sub.loc[(sub['time']>='2022-05-01 01:00:00')&(sub['time']<'2022-05-08 01:00:00'),1:] = sub1


sub1 = (train2.iloc[-24*7*1:,1:-1].values + train2.iloc[-24*7*2:-24*7*1,1:-1].values + \
        train2.iloc[-24*7*3:-24*7*2,1:-1].values)/3
sub.loc[(sub['time']>='2022-06-01 01:00:00')&(sub['time']<'2022-06-08 01:00:00'),1:] = sub1


sub1 = (train3.iloc[-24*7*1:,1:-1].values + train3.iloc[-24*7*2:-24*7*1,1:-1].values + \
        train3.iloc[-24*7*3:-24*7*2,1:-1].values + train3.iloc[-24*7*4:-24*7*3,1:-1].values)/4
sub.loc[(sub['time']>='2022-07-21 01:00:00')&(sub['time']<'2022-07-28 01:00:00'),1:] = sub1


sub1 = (train4.iloc[-24*7*1:,1:-1].values + train4.iloc[-24*7*2:-24*7*1,1:-1].values + \
        train4.iloc[-24*7*3:-24*7*2,1:-1].values)/3
sub.loc[(sub['time']>='2022-08-21 01:00:00')&(sub['time']<'2022-08-28 01:00:00'),1:] = sub1

In [8]:
sub.to_csv('use_4week_mean_0927.csv',index = False)  # 线上 0.58576808

In [9]:
sub.describe()

,flow_1,flow_2,flow_3,flow_4,flow_5,flow_6,flow_7,flow_8,flow_9,flow_10,flow_11,flow_12,flow_13,flow_14,flow_15,flow_16,flow_17,flow_18,flow_19,flow_20
count,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000,672.000000
mean,45.012946,20.277071,79.188889,47.040042,5.616270,86.000707,15.101885,2.314162,6.914993,2.797938,5.628361,3.491503,3.522991,3.165613,2.104204,5.393006,9.198090,7.035927,3.039922,2.765191
std,19.635147,11.862538,27.262591,21.932761,3.141802,43.027140,9.888814,1.499258,4.430748,2.025283,3.574436,2.383215,1.750024,2.301408,1.332759,3.392007,5.743931,4.233395,2.086353,1.419943
min,11.300000,2.525000,26.550000,11.500000,0.833333,12.933333,1.600000,0.166667,0.625000,0.130667,0.466667,0.224250,0.250000,0.166667,0.100000,0.575000,0.750000,0.661750,0.200667,0.166667
25%,30.966667,11.893750,62.608333,27.566667,3.131250,57.500000,7.272917,1.150000,3.572917,1.257833,2.858333,1.799667,2.518750,1.466667,1.125000,2.762500,4.631250,3.835813,1.545792,1.833333
50%,44.325000,19.566667,80.062500,48.116667,5.333333,87.512500,13.625000,2.033333,6.287500,2.600000,5.333333,3.082083,3.366667,2.750000,1.950000,5.029167,8.612500,6.664667,2.572250,2.800000
75%,55.908333,26.612500,94.868750,62.318750,7.733333,113.181250,21.200000,3.281250,9.500000,3.856750,7.366667,4.733000,4.527083,4.154167,2.927083,7.141667,12.866667,9.081333,4.242833,3.600000
max,112.050000,51.325000,186.550000,100.125000,13.166667,201.866667,42.075000,6.575000,20.000000,8.801500,15.950000,10.480000,9.066667,9.633333,6.000000,14.825000,24.600000,18.196000,9.979333,6.633333
